In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from pathlib import Path
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/lib'))
sys.path.append(str('/home/taoliu/Data/NAS-Projects-master/configs'))
from config_utils import load_config, dict2config, configure2str
from datasets     import get_datasets, SearchDataset
from procedures   import prepare_seed, prepare_logger, save_checkpoint, copy_checkpoint, get_optim_scheduler
from utils        import get_model_infos, obtain_accuracy
from log_utils    import AverageMeter, time_string, convert_secs2time
from models       import get_cell_based_tiny_net, get_search_spaces
from nas_102_api  import NASBench102API as API

/data/taoliu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
parser = argparse.ArgumentParser("GDAS")
parser.add_argument('--data_path',          type=str,   default= '/data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py', help='Path to dataset')
parser.add_argument('--dataset',            type=str,   default= 'cifar10', choices=['cifar10', 'cifar100', 'ImageNet16-120'], help='Choose between Cifar10/100 and ImageNet-16.')
# channels and number-of-cells
parser.add_argument('--search_space_name',  type=str,   default= 'nas-bench-102',help='The search space name.')
parser.add_argument('--max_nodes',          type=int,   default= 4 ,help='The maximum number of nodes.')
parser.add_argument('--channel',            type=int,   default= 16,help='The number of channels.')
parser.add_argument('--num_cells',          type=int,   default= 5, help='The number of cells in one stage.')
parser.add_argument('--track_running_stats',type=int,   default= 1,choices=[0,1],help='Whether use track_running_stats or not in the BN layer.')
parser.add_argument('--config_path',        type=str,   default= '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/GDAS.config'  ,help='The path of the configuration.')
# architecture leraning rate
parser.add_argument('--arch_learning_rate', type=float, default=3e-4, help='learning rate for arch encoding')
parser.add_argument('--arch_weight_decay',  type=float, default=1e-3, help='weight decay for arch encoding')
parser.add_argument('--tau_min',            type=float, default=0.1, help='The minimum tau for Gumbel')
parser.add_argument('--tau_max',            type=float, default=10, help='The maximum tau for Gumbel')
# log
parser.add_argument('--workers',            type=int,   default=2,    help='number of data loading workers (default: 2)')
parser.add_argument('--save_dir',           type=str,   default='./GDAS_Compensate_0.2_seed0/output/search-cell-nas-bench-102-cifar10',help='Folder to save checkpoints and log.')
parser.add_argument('--arch_nas_dataset',   type=str,   default='/home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth',help='The path to load the architecture dataset (tiny-nas-benchmark).')
parser.add_argument('--print_freq',         type=int,   default=200,help='print frequency (default: 200)')
parser.add_argument('--rand_seed',          type=int,   default=0, help='manual seed')
args = parser.parse_args([])

In [3]:
def cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts):
    compen_index   = cur_index
    for i in range(pre_index.shape[0]):
        if pre_index[i][0].item()==cur_index[i][0].item():
            a=np.random.randint(5)
            while a==pre_index[i][0].item():
                a=np.random.randint(5)
            compen_index[i][0]=a              
    one_h   = torch.zeros_like(cur_hardwts).scatter_(-1, compen_index, 1.0)
    compen_hardwts = one_h - cur_hardwts.detach() + cur_hardwts
    
    return compen_index,compen_hardwts
                
            

def search_func(xloader, network, criterion, scheduler, w_optimizer, a_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()

   # pre_index=0
 #   pre_hardwts=0
    
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(xloader):
      
        scheduler.update(None, 1.0 * step / len(xloader))        
           
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
    # measure data loading time
        data_time.update(time.time() - end)
    
    # update the weights
        w_optimizer.zero_grad()
                        
        cur_index,cur_hardwts=network.module.get_index_hardwts()        
                      
        if step==0:
            pre_index=cur_index
            pre_hardwts=cur_hardwts
            
            
        
        #network.module.set_genotype(pre_index) 
        _, logits,_,_ = network(base_inputs,1,pre_index,pre_hardwts)
        pre_loss = criterion(logits, base_targets).item()   
        
        com_index,com_hardwts=cal_compensate(pre_index,cur_index,pre_hardwts,cur_hardwts)
        
        #network.module.set_genotype(com_index)   
        _, logits,_,_ = network(base_inputs,1,com_index,com_hardwts)
        compen_loss = criterion(logits, base_targets).item()
        
        _, logits,cur_index,cur_hardwts = network(base_inputs,0,0,0)
        cur_loss = criterion(logits, base_targets)    
        
        base_loss =0.2*cur_loss+0.4*pre_loss+0.4*compen_loss
        
        base_loss.backward()
        torch.nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
    # record
        base_prec1, base_prec5 = obtain_accuracy(logits.data, base_targets.data, topk=(1, 5))
        base_losses.update(base_loss.item(),  base_inputs.size(0))
        base_top1.update  (base_prec1.item(), base_inputs.size(0))
        base_top5.update  (base_prec5.item(), base_inputs.size(0))
        
    ####current arch to previous arch    
        #network.module.set_genotype(cur_index)  
        pre_index=cur_index
        pre_hardwts=cur_hardwts
        
        
    # update the architecture-weight
        a_optimizer.zero_grad()
        _, logits,_,_ = network(arch_inputs,0,0,0)
        arch_loss = criterion(logits, arch_targets)
        arch_loss.backward()
        a_optimizer.step()
    # record
        arch_prec1, arch_prec5 = obtain_accuracy(logits.data, arch_targets.data, topk=(1, 5))
        arch_losses.update(arch_loss.item(),  arch_inputs.size(0))
        arch_top1.update  (arch_prec1.item(), arch_inputs.size(0))
        arch_top5.update  (arch_prec5.item(), arch_inputs.size(0))

    # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = '*SEARCH* ' + time_string() + ' [{:}][{:03d}/{:03d}]'.format(epoch_str, step, len(xloader))
            Tstr = 'Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})'.format(batch_time=batch_time, data_time=data_time)
            Wstr = 'Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=base_losses, top1=base_top1, top5=base_top5)
            Astr = 'Arch [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]'.format(loss=arch_losses, top1=arch_top1, top5=arch_top5)
            logger.log(Sstr + ' ' + Tstr + ' ' + Wstr + ' ' + Astr)
    return base_losses.avg, base_top1.avg, base_top5.avg, arch_losses.avg, arch_top1.avg, arch_top5.avg


In [4]:
#def main(xargs):
xargs=args
assert torch.cuda.is_available(), 'CUDA is not available.'
torch.backends.cudnn.enabled   = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads( xargs.workers )
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

train_data, _, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
if xargs.dataset == 'cifar10' or xargs.dataset == 'cifar100':
    split_Fpath = '/home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/cifar-split.txt'
    cifar_split = load_config(split_Fpath, None, None)
    train_split, valid_split = cifar_split.train, cifar_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
elif xargs.dataset.startswith('ImageNet16'):
    split_Fpath = 'configs/nas-benchmark/{:}-split.txt'.format(xargs.dataset)
    imagenet16_split = load_config(split_Fpath, None, None)
    train_split, valid_split = imagenet16_split.train, imagenet16_split.valid
    logger.log('Load split file from {:}'.format(split_Fpath))
else:
    raise ValueError('invalid dataset : {:}'.format(xargs.dataset))
  #config_path = 'configs/nas-benchmark/algos/GDAS.config'
config = load_config(xargs.config_path, {'class_num': class_num, 'xshape': xshape}, logger)
search_data   = SearchDataset(xargs.dataset, train_data, train_split, valid_split)
# data loader
search_loader = torch.utils.data.DataLoader(search_data, batch_size=config.batch_size, shuffle=True , num_workers=xargs.workers, pin_memory=True)
logger.log('||||||| {:10s} ||||||| Search-Loader-Num={:}, batch size={:}'.format(xargs.dataset, len(search_loader), config.batch_size))
logger.log('||||||| {:10s} ||||||| Config={:}'.format(xargs.dataset, config))

search_space = get_search_spaces('cell', xargs.search_space_name)
model_config = dict2config({'name': 'GDAS', 'C': xargs.channel, 'N': xargs.num_cells,
                          'max_nodes': xargs.max_nodes, 'num_classes': class_num,
                          'space'    : search_space,
                          'affine'   : False, 'track_running_stats': bool(xargs.track_running_stats)}, None)
search_model = get_cell_based_tiny_net(model_config)
#from models.cell_searchs.search_model_gdas import TinyNetworkGDAS as get_cell_based_tiny_net
#search_model = get_cell_based_tiny_net(xargs.channel,xargs.num_cells,xargs.max_nodes,class_num,search_space, False,bool(xargs.track_running_stats))
logger.log('search-model :\n{:}'.format(search_model))

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.get_weights(), config)
a_optimizer = torch.optim.Adam(search_model.get_alphas(), lr=xargs.arch_learning_rate, betas=(0.5, 0.999), weight_decay=xargs.arch_weight_decay)
logger.log('w-optimizer : {:}'.format(w_optimizer))
logger.log('a-optimizer : {:}'.format(a_optimizer))
logger.log('w-scheduler : {:}'.format(w_scheduler))
logger.log('criterion   : {:}'.format(criterion))
#flop, param  = get_model_infos(search_model, xshape)
#logger.log('{:}'.format(search_model))
#logger.log('FLOP = {:.2f} M, Params = {:.2f} MB'.format(flop, param))
logger.log('search-space : {:}'.format(search_space))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log('{:} create API = {:} done'.format(time_string(), api))

last_info, model_base_path, model_best_path = logger.path('info'), logger.path('model'), logger.path('best')
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists(): # automatically resume from previous checkpoint
    logger.log("=> loading checkpoint of the last-info '{:}' start".format(last_info))
    last_info   = torch.load(last_info)
    start_epoch = last_info['epoch']
    checkpoint  = torch.load(last_info['last_checkpoint'])
    genotypes   = checkpoint['genotypes']
    valid_accuracies = checkpoint['valid_accuracies']
    search_model.load_state_dict( checkpoint['search_model'] )
    w_scheduler.load_state_dict ( checkpoint['w_scheduler'] )
    w_optimizer.load_state_dict ( checkpoint['w_optimizer'] )
    a_optimizer.load_state_dict ( checkpoint['a_optimizer'] )
    logger.log("=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(last_info, start_epoch))
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {'best': -1}, {}

  # start training
start_time, search_time, epoch_time, total_epoch = time.time(), AverageMeter(), AverageMeter(), config.epochs + config.warmup
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = 'Time Left: {:}'.format( convert_secs2time(epoch_time.val * (total_epoch-epoch), True) )
    epoch_str = '{:03d}-{:03d}'.format(epoch, total_epoch)
    search_model.set_tau( xargs.tau_max - (xargs.tau_max-xargs.tau_min) * epoch / (total_epoch-1) )
    logger.log('\n[Search the {:}-th epoch] {:}, tau={:}, LR={:}'.format(epoch_str, need_time, search_model.get_tau(), min(w_scheduler.get_lr())))

    search_w_loss, search_w_top1, search_w_top5, valid_a_loss , valid_a_top1 , valid_a_top5 \
              = search_func(search_loader, network, criterion, w_scheduler, w_optimizer, a_optimizer, epoch_str, xargs.print_freq, logger)
    search_time.update(time.time() - start_time)
    logger.log('[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s'.format(epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum))
    logger.log('[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%'.format(epoch_str, valid_a_loss , valid_a_top1 , valid_a_top5 ))
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies['best']:
        valid_accuracies['best'] = valid_a_top1
        genotypes['best']        = search_model.genotype()
        find_best = True
    else: find_best = False

    genotypes[epoch] = search_model.genotype()
    logger.log('<<<--->>> The {:}-th epoch : {:}'.format(epoch_str, genotypes[epoch]))
    # save checkpoint
    save_path = save_checkpoint({'epoch' : epoch + 1,
                'args'  : deepcopy(xargs),
                'search_model': search_model.state_dict(),
                'w_optimizer' : w_optimizer.state_dict(),
                'a_optimizer' : a_optimizer.state_dict(),
                'w_scheduler' : w_scheduler.state_dict(),
                'genotypes'   : genotypes,
                'valid_accuracies' : valid_accuracies},
                model_base_path, logger)
    last_info = save_checkpoint({
          'epoch': epoch + 1,
          'args' : deepcopy(args),
          'last_checkpoint': save_path,
          }, logger.path('info'), logger)
    if find_best:
        logger.log('<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.'.format(epoch_str, valid_a_top1))
        copy_checkpoint(model_base_path, model_best_path, logger)
    with torch.no_grad():
        logger.log('arch-parameters :\n{:}'.format( nn.functional.softmax(search_model.arch_parameters, dim=-1).cpu() ))
    if api is not None: logger.log('{:}'.format(api.query_by_arch( genotypes[epoch] )))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log('\n' + '-'*100)
  # check the performance from the architecture dataset
logger.log('GDAS : run {:} epochs, cost {:.1f} s, last-geno is {:}.'.format(total_epoch, search_time.sum, genotypes[total_epoch-1]))
if api is not None: logger.log('{:}'.format( api.query_by_arch(genotypes[total_epoch-1]) ))
logger.close()

Main Function with logger : Logger(dir=GDAS_Compensate_0.2_seed0/output/search-cell-nas-bench-102-cifar10, use-tf=False, writer=None)
Arguments : -------------------------------
arch_learning_rate : 0.0003
arch_nas_dataset : /home/taoliu/Data/NAS-Projects-master/NAS-Bench-102-v1_0-e61699.pth
arch_weight_decay : 0.001
channel          : 16
config_path      : /home/taoliu/Data/NAS-Projects-master/configs/nas-benchmark/algos/GDAS.config
data_path        : /data/taoliu/ENNAS_benchmark102/data/cifar-10-batches-py
dataset          : cifar10
max_nodes        : 4
num_cells        : 5
print_freq       : 200
rand_seed        : 0
save_dir         : ./GDAS_Compensate_0.2_seed0/output/search-cell-nas-bench-102-cifar10
search_space_name : nas-bench-102
tau_max          : 10
tau_min          : 0.1
track_running_stats : 1
workers          : 2
Python  Version  : 3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56)  [GCC 7.2.0]
Pillow  Version  : 5.1.0
PyTorch Version  : 0.4.1
cuDNN   Version  : 7102

[2020-01-18 04:46:27] create API = NASBench102API(15625/15625 architectures) done
=> do not find the last-info file : GDAS_Compensate_0.2_seed0/output/search-cell-nas-bench-102-cifar10/seed-0-last-info.pth

[Search the 000-250-th epoch] Time Left: [00:00:00], tau=10.0, LR=0.025
*SEARCH* [2020-01-18 04:46:30] [000-250][000/391] Time 1.10 (1.10) Data 0.50 (0.50) Base [Loss 2.298 (2.298)  Prec@1 18.75 (18.75) Prec@5 62.50 (62.50)] Arch [Loss 2.333 (2.333)  Prec@1 7.81 (7.81) Prec@5 46.88 (46.88)]
*SEARCH* [2020-01-18 04:48:14] [000-250][200/391] Time 0.51 (0.52) Data 0.00 (0.00) Base [Loss 2.268 (2.289)  Prec@1 20.31 (11.79) Prec@5 70.31 (53.10)] Arch [Loss 2.169 (2.289)  Prec@1 21.88 (11.71) Prec@5 70.31 (54.07)]
*SEARCH* [2020-01-18 04:50:08] [000-250][390/391] Time 0.96 (0.56) Data 0.00 (0.00) Base [Loss 2.271 (2.270)  Prec@1 2.50 (13.10) Prec@5 60.00 (56.43)] Arch [Loss 2.258 (2.273)  Prec@1 22.50 (13.02) Prec@5 60.00 (56.46)]
[000-250] searching : loss=2.27, accuracy@1=13.10%, accura

*SEARCH* [2020-01-18 05:09:01] [003-250][000/391] Time 2.45 (2.45) Data 0.97 (0.97) Base [Loss 2.068 (2.068)  Prec@1 25.00 (25.00) Prec@5 81.25 (81.25)] Arch [Loss 2.234 (2.234)  Prec@1 21.88 (21.88) Prec@5 67.19 (67.19)]
*SEARCH* [2020-01-18 05:14:02] [003-250][200/391] Time 1.53 (1.51) Data 0.00 (0.01) Base [Loss 2.086 (2.154)  Prec@1 25.00 (18.00) Prec@5 75.00 (68.63)] Arch [Loss 2.089 (2.162)  Prec@1 18.75 (17.48) Prec@5 75.00 (67.98)]
*SEARCH* [2020-01-18 05:18:46] [003-250][390/391] Time 1.40 (1.50) Data 0.00 (0.00) Base [Loss 2.084 (2.144)  Prec@1 15.00 (18.02) Prec@5 80.00 (69.39)] Arch [Loss 2.107 (2.147)  Prec@1 25.00 (18.01) Prec@5 72.50 (69.19)]
[003-250] searching : loss=2.14, accuracy@1=18.02%, accuracy@5=69.39%, time-cost=1936.1 s
[003-250] evaluate  : loss=2.15, accuracy@1=18.01%, accuracy@5=69.19%
<<<--->>> The 003-250-th epoch : Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|none~1|+|skip_connect~0|skip_connect~1|avg_pool_3x3~2|)
Find GDAS_Compensate_0.2_seed

*SEARCH* [2020-01-18 05:38:26] [006-250][000/391] Time 2.00 (2.00) Data 0.51 (0.51) Base [Loss 2.154 (2.154)  Prec@1 17.19 (17.19) Prec@5 67.19 (67.19)] Arch [Loss 2.257 (2.257)  Prec@1 20.31 (20.31) Prec@5 64.06 (64.06)]
*SEARCH* [2020-01-18 05:43:23] [006-250][200/391] Time 1.59 (1.49) Data 0.00 (0.00) Base [Loss 2.083 (2.080)  Prec@1 20.31 (20.41) Prec@5 73.44 (73.87)] Arch [Loss 2.333 (2.097)  Prec@1 6.25 (19.80) Prec@5 46.88 (73.09)]
*SEARCH* [2020-01-18 05:48:09] [006-250][390/391] Time 1.47 (1.50) Data 0.00 (0.00) Base [Loss 2.228 (2.081)  Prec@1 20.00 (20.49) Prec@5 72.50 (73.65)] Arch [Loss 2.151 (2.089)  Prec@1 20.00 (20.23) Prec@5 67.50 (73.70)]
[006-250] searching : loss=2.08, accuracy@1=20.49%, accuracy@5=73.65%, time-cost=3697.4 s
[006-250] evaluate  : loss=2.09, accuracy@1=20.23%, accuracy@5=73.70%
<<<--->>> The 006-250-th epoch : Structure(4 nodes with |skip_connect~0|+|avg_pool_3x3~0|none~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|)
Find GDAS_Compensate_0.2_seed0

*SEARCH* [2020-01-18 06:07:41] [009-250][000/391] Time 1.89 (1.89) Data 0.51 (0.51) Base [Loss 2.002 (2.002)  Prec@1 26.56 (26.56) Prec@5 89.06 (89.06)] Arch [Loss 1.933 (1.933)  Prec@1 25.00 (25.00) Prec@5 78.12 (78.12)]
*SEARCH* [2020-01-18 06:12:41] [009-250][200/391] Time 1.49 (1.50) Data 0.00 (0.00) Base [Loss 2.228 (2.036)  Prec@1 17.19 (21.73) Prec@5 78.12 (76.83)] Arch [Loss 2.111 (2.038)  Prec@1 20.31 (21.13) Prec@5 76.56 (76.55)]
*SEARCH* [2020-01-18 06:17:26] [009-250][390/391] Time 1.50 (1.50) Data 0.00 (0.00) Base [Loss 1.977 (2.032)  Prec@1 12.50 (21.86) Prec@5 82.50 (77.07)] Arch [Loss 2.014 (2.034)  Prec@1 27.50 (21.28) Prec@5 82.50 (76.83)]
[009-250] searching : loss=2.03, accuracy@1=21.86%, accuracy@5=77.07%, time-cost=5454.2 s
[009-250] evaluate  : loss=2.03, accuracy@1=21.28%, accuracy@5=76.83%
<<<--->>> The 009-250-th epoch : Structure(4 nodes with |skip_connect~0|+|skip_connect~0|none~1|+|avg_pool_3x3~0|skip_connect~1|avg_pool_3x3~2|)
Find GDAS_Compensate_0.2_seed

...

*SEARCH* [2020-01-20 03:29:45] [237-250][000/391] Time 2.73 (2.73) Data 0.52 (0.52) Base [Loss 1.187 (1.187)  Prec@1 71.88 (71.88) Prec@5 96.88 (96.88)] Arch [Loss 1.009 (1.009)  Prec@1 73.44 (73.44) Prec@5 96.88 (96.88)]
*SEARCH* [2020-01-20 03:39:09] [237-250][200/391] Time 2.35 (2.82) Data 0.00 (0.00) Base [Loss 0.787 (0.892)  Prec@1 76.56 (71.35) Prec@5 100.00 (97.03)] Arch [Loss 1.166 (0.898)  Prec@1 60.94 (68.85) Prec@5 95.31 (96.83)]
*SEARCH* [2020-01-20 03:48:07] [237-250][390/391] Time 3.00 (2.83) Data 0.00 (0.00) Base [Loss 0.736 (0.892)  Prec@1 67.50 (71.35) Prec@5 97.50 (97.15)] Arch [Loss 0.839 (0.907)  Prec@1 72.50 (68.82) Prec@5 97.50 (96.42)]
[237-250] searching : loss=0.89, accuracy@1=71.35%, accuracy@5=97.15%, time-cost=169226.4 s
[237-250] evaluate  : loss=0.91, accuracy@1=68.82%, accuracy@5=96.42%
<<<--->>> The 237-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_1x1~1|nor_conv_1x1~2|)
Find GDAS_Compensate_0.2_s

*SEARCH* [2020-01-20 04:54:26] [240-250][000/391] Time 4.95 (4.95) Data 0.98 (0.98) Base [Loss 0.623 (0.623)  Prec@1 82.81 (82.81) Prec@5 100.00 (100.00)] Arch [Loss 1.010 (1.010)  Prec@1 65.62 (65.62) Prec@5 96.88 (96.88)]
*SEARCH* [2020-01-20 05:11:04] [240-250][200/391] Time 4.37 (4.99) Data 0.00 (0.01) Base [Loss 0.931 (0.837)  Prec@1 71.88 (71.25) Prec@5 98.44 (97.33)] Arch [Loss 0.770 (0.843)  Prec@1 68.75 (71.06) Prec@5 98.44 (97.22)]
*SEARCH* [2020-01-20 05:23:08] [240-250][390/391] Time 3.18 (4.42) Data 0.00 (0.00) Base [Loss 0.615 (0.856)  Prec@1 87.50 (71.48) Prec@5 100.00 (97.13)] Arch [Loss 1.153 (0.852)  Prec@1 60.00 (70.62) Prec@5 97.50 (97.25)]
[240-250] searching : loss=0.86, accuracy@1=71.48%, accuracy@5=97.13%, time-cost=174926.4 s
[240-250] evaluate  : loss=0.85, accuracy@1=70.62%, accuracy@5=97.25%
<<<--->>> The 240-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_1x1~1|nor_conv_1x1~2|)
Find GDAS_Compensate_0.2

*SEARCH* [2020-01-20 06:12:45] [243-250][000/391] Time 3.94 (3.94) Data 0.52 (0.52) Base [Loss 0.912 (0.912)  Prec@1 78.12 (78.12) Prec@5 100.00 (100.00)] Arch [Loss 0.735 (0.735)  Prec@1 76.56 (76.56) Prec@5 100.00 (100.00)]
*SEARCH* [2020-01-20 06:25:33] [243-250][200/391] Time 4.33 (3.84) Data 0.00 (0.00) Base [Loss 1.006 (0.858)  Prec@1 76.56 (71.79) Prec@5 100.00 (96.85)] Arch [Loss 1.096 (0.872)  Prec@1 67.19 (69.92) Prec@5 98.44 (96.88)]
*SEARCH* [2020-01-20 06:36:21] [243-250][390/391] Time 2.97 (3.63) Data 0.00 (0.00) Base [Loss 1.137 (0.842)  Prec@1 72.50 (72.49) Prec@5 97.50 (97.22)] Arch [Loss 0.895 (0.864)  Prec@1 70.00 (70.24) Prec@5 95.00 (97.11)]
[243-250] searching : loss=0.84, accuracy@1=72.49%, accuracy@5=97.22%, time-cost=179318.3 s
[243-250] evaluate  : loss=0.86, accuracy@1=70.24%, accuracy@5=97.11%
<<<--->>> The 243-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_1x1~1|nor_conv_1x1~2|)
Find GDAS_Compensate_0

*SEARCH* [2020-01-20 07:10:30] [246-250][000/391] Time 2.03 (2.03) Data 0.51 (0.51) Base [Loss 0.982 (0.982)  Prec@1 7.81 (7.81) Prec@5 45.31 (45.31)] Arch [Loss 1.954 (1.954)  Prec@1 29.69 (29.69) Prec@5 85.94 (85.94)]
*SEARCH* [2020-01-20 07:16:08] [246-250][200/391] Time 1.98 (1.70) Data 0.00 (0.00) Base [Loss 0.717 (0.842)  Prec@1 84.38 (72.63) Prec@5 100.00 (97.08)] Arch [Loss 1.109 (0.885)  Prec@1 64.06 (69.99) Prec@5 95.31 (96.53)]
*SEARCH* [2020-01-20 07:23:20] [246-250][390/391] Time 1.91 (1.98) Data 0.00 (0.00) Base [Loss 0.716 (0.844)  Prec@1 70.00 (72.63) Prec@5 100.00 (97.46)] Arch [Loss 0.736 (0.854)  Prec@1 75.00 (70.98) Prec@5 100.00 (97.00)]
[246-250] searching : loss=0.84, accuracy@1=72.63%, accuracy@5=97.46%, time-cost=182136.6 s
[246-250] evaluate  : loss=0.85, accuracy@1=70.98%, accuracy@5=97.00%
<<<--->>> The 246-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_1x1~1|nor_conv_3x3~2|)
Find GDAS_Compensate_0.2_s

*SEARCH* [2020-01-20 07:53:15] [249-250][000/391] Time 2.94 (2.94) Data 0.51 (0.51) Base [Loss 0.795 (0.795)  Prec@1 71.88 (71.88) Prec@5 96.88 (96.88)] Arch [Loss 0.917 (0.917)  Prec@1 71.88 (71.88) Prec@5 93.75 (93.75)]
*SEARCH* [2020-01-20 08:00:53] [249-250][200/391] Time 1.83 (2.29) Data 0.00 (0.00) Base [Loss 1.201 (0.837)  Prec@1 62.50 (72.71) Prec@5 98.44 (97.60)] Arch [Loss 0.587 (0.840)  Prec@1 75.00 (71.22) Prec@5 98.44 (97.47)]
*SEARCH* [2020-01-20 08:07:44] [249-250][390/391] Time 1.48 (2.23) Data 0.00 (0.00) Base [Loss 0.813 (0.839)  Prec@1 70.00 (72.27) Prec@5 97.50 (97.63)] Arch [Loss 1.073 (0.864)  Prec@1 67.50 (70.32) Prec@5 100.00 (96.73)]
[249-250] searching : loss=0.84, accuracy@1=72.27%, accuracy@5=97.63%, time-cost=184800.1 s
[249-250] evaluate  : loss=0.86, accuracy@1=70.32%, accuracy@5=96.73%
<<<--->>> The 249-250-th epoch : Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_3x3~0|none~1|+|nor_conv_3x3~0|nor_conv_1x1~1|nor_conv_3x3~2|)
Find GDAS_Compensate_0.2_s